# Create and run model

## Load libraries

In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, auc, confusion_matrix, f1_score

import pandas as pd
import numpy as np

## Set global parameters

In [2]:
data_path = '../data/03_primary/processed_data.csv'
artifact_path = "../data/06_models/"

test_size = 0.2
random_state = 18
cost_fn = 10
cost_fp = 1

## Load data

In [3]:
# Load data
raw_data = pd.read_csv(data_path)
raw_data = raw_data.astype('float32')


In [4]:
data = raw_data[['PAYMENT_RATE', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_BIRTH', 'EXT_SOURCE_1', 'DAYS_EMPLOYED', 'DAYS_EMPLOYED_PERC', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'ANNUITY_INCOME_PERC', 'INSTAL_DBD_MEAN', 'AMT_ANNUITY', 'TARGET']].astype('float32')

In [5]:

# Remove lines with TARGET = NaN
data = data.dropna(subset=["TARGET"])

## Split data

In [6]:
X = data.drop("TARGET", axis=1)
y = data["TARGET"]

In [7]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

## Create model

### Train and search best parameters of model

In [9]:
from lightgbm import LGBMClassifier

In [8]:
# create param grid with 3 options on each for LGBMClassifier
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5],
    'num_leaves': [31, 63, 127],
    'boosting_type': ['gbdt', 'dart', 'goss']
}

In [10]:
def train_model(X_train, y_train, param_grid):
    # Initialisation du modèle
    model = LGBMClassifier()

    # Initialisation de GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

    # Entraînement du modèle
    grid_search.fit(X_train, y_train)

    return grid_search

## Evaluate model

In [11]:
# Define a work cost function
def cost_metric(y_true, y_pred, cost_fn=cost_fn, cost_fp=cost_fp):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cost = (fn * cost_fn) + (fp * cost_fp)
    return cost

In [12]:
def optimize_threshold(y_true, y_proba, cost_fn=cost_fn, cost_fp=cost_fp):
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    costs = []
    for threshold in thresholds:
        y_pred = (y_proba >= threshold).astype(int)
        cost = cost_metric(y_true, y_pred, cost_fn, cost_fp)
        costs.append(cost)
    optimal_idx = np.argmin(costs)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold


In [13]:
def assess_model(best_model, X_test, y_test):
    # Prédictions sur l'ensemble de test
    y_proba = best_model.predict_proba(X_test)[:, 1]
    optimal_threshold = optimize_threshold(y_test, y_proba)
    y_pred = (y_proba >= optimal_threshold).astype(int)

    # Calcul des métriques
    auc_roc = roc_auc_score(y_test, y_proba)
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    auc_pr = auc(recall, precision) # Area Under the Precision-Recall Curve
    cost = cost_metric(y_test, y_pred)
    f1 = f1_score(y_test, y_pred) # F1 score
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy, auc_roc, cost, auc_pr, f1, optimal_threshold

# MLFlow

## Import libraries

In [14]:
import mlflow
from mlflow.models import infer_signature

## Set experiment and tracking URI

In [15]:
mlflow.set_tracking_uri(uri="../mlruns")
mlflow.set_experiment("Credit_Scoring_Model")

2024/08/31 13:29:48 INFO mlflow.tracking.fluent: Experiment with name 'Credit_Scoring_Model' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/Z478SG/Desktop/Ecole/OpenClassrooms-Projet-7/modeling/notebooks/../mlruns/856765195761166768', creation_time=1725103788172, experiment_id='856765195761166768', last_update_time=1725103788172, lifecycle_stage='active', name='Credit_Scoring_Model', tags={}>

## Log experiment data into MLFlow

### Log data

In [16]:
# Start an MLflow run
with mlflow.start_run():
    ## Run experiment
    # Train model
    grid_search = train_model(X_train, y_train, param_grid)

    best_model = grid_search.best_estimator_

    # Assess model
    accuracy, auc_roc, cost, auc_pr, f1, optimal_threshold = assess_model(best_model, X_test, y_test)

    ## Log data
    # Set a tag that (key, value) we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "LGMB model for credit scoring data, 20% test split")

    # Log parameters
    mlflow.log_params({
        "test_size": test_size,
        "random_state": random_state,
        "cost_fn": cost_fn,
        "cost_fp": cost_fp,
        })
    grid_params_with_suffix = {f"{k}_tested": v for k, v in param_grid.items()}
    mlflow.log_params(grid_params_with_suffix)

    mlflow.log_params(grid_search.best_params_)

    # Log metrics (accuracy and AUC-ROC)
    mlflow.log_metrics({
        "Accuracy": accuracy,
        "AUC-ROC": auc_roc,
        "Cost": cost,
        "AUC-PR": auc_pr,
        "F1": f1,
        "Optimal Threshold": optimal_threshold,
        })

    # Infer the model signature (input and output schema) from the training data
    signature = infer_signature(X_train, best_model.predict(X_train))
    
    # Log model
    model_info = mlflow.sklearn.log_model(
        sk_model=best_model, 
        artifact_path=artifact_path, 
        signature=signature,
        input_example=X_train,
        )

    # Log data path
    mlflow.log_artifact(data_path)
    

[LightGBM] [Info] Number of positive: 19945, number of negative: 226060
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022565 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3060
[LightGBM] [Info] Number of data points in the train set: 246005, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081076 -> initscore=-2.427822
[LightGBM] [Info] Start training from score -2.427822


MlflowException: Changing param values is not allowed. Param with key='boosting_type' was already logged with value='['gbdt', 'dart', 'goss']' for run ID='59008a69decb40259038e147704fc773'. Attempted logging new value 'gbdt'.

### View experiments in MLFlow UI

In a background terminal, run:  

./run_mlflow_ui.py

### If result is good : Register the model as an official model

In [ ]:
# If not registered during the run, officially register the model artifact with a specific name

result = mlflow.register_model(
    model_info.model_uri, "LGBMClassifier-V1"
)